In [84]:
%run UTILS.ipynb

import librosa
import numpy as np

In [85]:
audio_files = [
    'AUDIO.wav',
    'FORMANT.csv',
    'COVAREP.csv'
]

df_audio_files = retrieve_files(base_directory, audio_files, num_folders=1)
df_audio_files

,ID,AUDIO,FORMANT,COVAREP
0,475,data/475_P/475_AUDIO.wav,data/475_P/475_FORMANT.csv,data/475_P/475_COVAREP.csv


In [86]:
def add_timestamps(df, sampling_rate=100):
    # Calculate time interval based on the sampling rate
    time_interval = 1 / sampling_rate
    
    # Generate timestamps based on the number of samples
    num_samples = len(df)
    timestamps = np.arange(0, num_samples * time_interval, time_interval)
    
    # Add the timestamps as a new column in the DataFrame
    df['TIMESTAMP'] = timestamps[:num_samples]
    
    return df

In [87]:
def preprocess_AUDIO(path):
    # this is incorrect, i need to turn this into a dataframe with correctly labled shitt
    y, sr = librosa.load(path, sr=None)

    # Extract various features
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    chroma = librosa.feature.chroma_stft(y=y, sr=sr)
    zcr = librosa.feature.zero_crossing_rate(y)

    # Aggregate the features
    mfccs_mean = pd.DataFrame(np.mean(mfccs, axis=1)).add_prefix('MFCCSmean_')
    chroma_mean = pd.DataFrame(np.mean(chroma, axis=1)).add_prefix('CHROMAmean_')
    zcr_mean = pd.DataFrame(np.mean(zcr, axis=1)).add_prefix('ZCRmean_')
    
    audio_features = pd.DataFrame()
    
    audio_features = pd.concat([audio_features, mfccs_mean], axis=1)
    audio_features = pd.concat([audio_features, chroma_mean], axis=1)
    audio_features = pd.concat([audio_features, zcr_mean], axis=1)
    
    return audio_features

def preprocess_FORMANT(path):
    formant_features = pd.read_csv(path, header=None)
    FORMANT_FEATURES = ['F1', 'F2', 'F3', 'F4', 'F5']
    formant_features.columns = FORMANT_FEATURES
    formant_features
    return formant_features


def preprocess_COVAREP(path):
    covarep_features = pd.read_csv(path, header=None)
    COVAREP_FEATURES = [
        'F0',
        'VUV',
        'NAQ',
        'QOQ',
        'H1H2',
        'PSP',
        'MDQ',
        'peakSlope',
        'Rd',
        'Rd_conf',
        'creak',

        'MCEP_0',
        'MCEP_1',
        'MCEP_2',
        'MCEP_3',
        'MCEP_4',
        'MCEP_5',
        'MCEP_6',
        'MCEP_7',
        'MCEP_8',
        'MCEP_9',
        'MCEP_10',
        'MCEP_11',
        'MCEP_12',
        'MCEP_13',
        'MCEP_14',
        'MCEP_15',
        'MCEP_16',
        'MCEP_17',
        'MCEP_18',
        'MCEP_19',
        'MCEP_20',
        'MCEP_21',
        'MCEP_22',
        'MCEP_23',
        'MCEP_24',

        'HMPDM_0',
        'HMPDM_1',
        'HMPDM_2',
        'HMPDM_3',
        'HMPDM_4',
        'HMPDM_5',
        'HMPDM_6',
        'HMPDM_7',
        'HMPDM_8',
        'HMPDM_9',
        'HMPDM_10',
        'HMPDM_11',
        'HMPDM_12',
        'HMPDM_13',
        'HMPDM_14',
        'HMPDM_15',
        'HMPDM_16',
        'HMPDM_17',
        'HMPDM_18',
        'HMPDM_19',
        'HMPDM_20',
        'HMPDM_21',
        'HMPDM_22',
        'HMPDM_23',
        'HMPDM_24',

        'HMPDD_0',
        'HMPDD_1',
        'HMPDD_2',
        'HMPDD_3',
        'HMPDD_4',
        'HMPDD_5',
        'HMPDD_6',
        'HMPDD_7',
        'HMPDD_8',  
        'HMPDD_9',
        'HMPDD_10',
        'HMPDD_11',
        'HMPDD_12',
    ]
    covarep_features.columns = COVAREP_FEATURES
    covarep_features
    return covarep_features

In [91]:
from IPython.display import clear_output

df_audio = pd.DataFrame()

for i in df_audio_files.index:
    df_concat = pd.DataFrame()
    
    # preprocess all the features
    #audio = preprocess_AUDIO(df_audio_files['AUDIO'][i]).add_prefix('AUDIO_') TODO: figure out if you can even get anything useful from audio
    formant = preprocess_FORMANT(df_audio_files['FORMANT'][i]).add_prefix('FORMANT_')
    covarep = preprocess_COVAREP(df_audio_files['COVAREP'][i]).add_prefix('COVAREP_')
 
    # concatenate all the features
    df_concat = pd.concat([ formant, covarep], axis=1) #TODO: add audio later

    # ensure the ID is a valid value and assign it as a column
    df_concat['ID'] = df_audio_files['ID'].iloc[i]
    
    # add PHQ binary and timestamps
    df_concat = append_PHQ_Binary(df_concat)
    df_concat = add_timestamps(df_concat)
    
    # append to the main DataFrame
    df_audio = df_audio._append(df_concat, ignore_index=True)
    
    clear_output()
    print("finished processing file", i) 

clear_output()
df_audio 


,FORMANT_F1,FORMANT_F2,FORMANT_F3,FORMANT_F4,FORMANT_F5,COVAREP_F0,COVAREP_VUV,COVAREP_NAQ,COVAREP_QOQ,COVAREP_H1H2,...,COVAREP_HMPDD_6,COVAREP_HMPDD_7,COVAREP_HMPDD_8,COVAREP_HMPDD_9,COVAREP_HMPDD_10,COVAREP_HMPDD_11,COVAREP_HMPDD_12,ID,PHQ_Binary,TIMESTAMP
0,738.36,1809.7,2605.3,3911.3,4616.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,475,0,0.00
1,788.51,1738.3,2327.8,2971.0,4256.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,475,0,0.01
2,800.84,1661.1,2231.4,2831.1,4090.1,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,475,0,0.02
3,765.59,1666.9,2526.0,3311.4,4298.3,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,475,0,0.03
4,800.46,1518.4,2609.0,3267.2,4465.6,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,475,0,0.04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58715,996.50,1836.0,2891.0,4082.0,4828.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,475,0,587.15
58716,937.50,1902.5,3031.5,3918.0,4687.5,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,475,0,587.16
58717,793.00,1848.0,3617.0,4019.5,4625.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,475,0,587.17
58718,793.00,1727.0,2957.0,3683.5,4425.5,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,475,0,587.18


In [96]:
# down-sampling the high frequency audio data to match the low frequency facial data for multi-modality
df_audio.set_index('TIMESTAMP', inplace=True)
pd.to_timedelta(df_audio.index , unit='s')
df_audio = df_audio.resample('33ms').mean()


KeyError: "None of ['TIMESTAMP'] are in the columns"